In [12]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd


from data_preprocessing import *
from model import *
from train import *
from utils import *

In [13]:
# pre pre-process images
# python preprocess.py

# Build dataset
# python -m tensorflow_datasets.scripts.download_and_prepare --datasets=mri_dataset --module_import=datasets.mri_dataset --manual_dir=data/processed --data_dir=data/

In [14]:
# img_path = os.path.join(data_folder, raw_folder, train_folder, '851.png')
# img = cv.imread(img_path)
# n_img = hist_norm(img)
# clahe_img1 = clahe(img, clipLimit=8, tileGridSize=(32, 32))
# clahe_img2 = clahe(img, clipLimit=8, tileGridSize=(8, 8))

# # n_clahe_img = clahe(n_img, clipLimit=clip, tileGridSize=tile_size)
# combined_imgs = np.hstack((img, clahe_img1, clahe_img2))
# plt.figure(figsize=(20, 20))
# plt.imshow(combined_imgs)

## GPU and Mixed Precision Setup

In [15]:
gpu_setup()
mixed_precision_setup()

## Load Dataset

In [16]:
import tensorflow_datasets as tfds
from datasets.mri_dataset import MriDataset
from tensorflow.data import Dataset
from tensorflow.keras.callbacks import TensorBoard


data_folder = 'data'
raw_folder = 'raw'
processed_folder = 'processed'
dataset = 'mri_dataset'
train_folder = 'train'
test_folder = 'test'
train_label = 'train_label.csv'

SEED = 0

experiment_name = 'MobileNetV2'
model_params = {
    'model_type': 'mobile',
    'head_type': 'standard',
    'image_shape': (512, 512, 3),
    'num_classes': 3,
}
base_hyperparams = {
    'train_batch_size': 64,
    'valid_batch_size': 64,
    'test_batch_size': 64,
    'num_epochs': 1,
    'learning_rate': 1e-4,
    'dropout': 0.2
}
fine_hyperparams = {
    'num_epochs': 1,
    'learning_rate': 1e-5,
    'fine_tune_at': 100,
}

tf.random.set_seed(SEED)

train_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[:{k}%]+train[{k+10}%:]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
valid_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[{k}%:{k+10}%]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
test_ds_raw, test_info_raw = tfds.load(
    name='mri_dataset', 
    split='test', 
    download=False, 
    shuffle_files=False, 
    as_supervised=False, 
    with_info=True,
    data_dir=data_folder
)

train_folds = [ preprocess(ds, model_params, batch_size=base_hyperparams['train_batch_size'], ds_type='train') for ds in train_folds ]
valid_folds = [ preprocess(ds, model_params, batch_size=base_hyperparams['valid_batch_size'], ds_type='valid') for ds in valid_folds ]
test_ds = preprocess(test_ds_raw, model_params, batch_size=base_hyperparams['test_batch_size'], ds_type='test')

In [17]:
train_valid_df = pd.read_csv(os.path.join(data_folder, processed_folder, train_label))
train_ds = train_folds[0]
valid_ds = valid_folds[0]

print(f'Number of train batches: {train_ds.cardinality()}')
print(f'Number of valid batches: {valid_ds.cardinality()}')
print(f'Number of test batches: {test_ds.cardinality()}')

Number of train batches: 17
Number of valid batches: 2
Number of test batches: 5


## Visualise Dataset

In [18]:
# # disable image resize and rescale in preprocess function ONLY for visualisation
# # Train Data
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(9):
#     for i in range(9):
#       ax = plt.subplot(3, 3, i + 1)
#       plt.imshow(images[i].numpy().astype("uint8"))
#       plt.title(labels[i].numpy())
#       plt.axis("off")

In [19]:
# # Test Data
# plt.figure(figsize=(10, 10))
# for i, ds in enumerate(test_ds_raw.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(ds['image'].numpy().astype("uint8"))
#     plt.title('ID: {}'.format(ds['id'].numpy()))
#     plt.axis("off")

In [9]:
# TODO: Fix visualisation
# data_augmentation = create_augmentation_layer()

# for image, _ in train_ds.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

# K-Fold Cross Validation

In [20]:
tf.random.set_seed(SEED)

models = cross_validate(experiment_name, train_folds, valid_folds, model_params, base_hyperparams, fine_hyperparams)
#     print('Saving model\n')
#     filename = os.path.join('models', experiment_name, '.h5')
#     model.save(filename)

: 0.9250307083129883 | Train Accuracy: 0.5677480697631836 | Validation Loss: 0.7722243666648865 | Validation Accuracy: 0.6896551847457886

# -------------------- MobileNetV2: 2-fold -------------------- #
17/17 [==============================] - 4s 241ms/step - loss: 1.1105 - accuracy: 0.4552 - val_loss: 1.1106 - val_accuracy: 0.4569
Number of layers in the base model:  155
Epoch 2/2
17/17 [==============================] - 5s 284ms/step - loss: 0.8541 - accuracy: 0.6441 - val_loss: 0.7785 - val_accuracy: 0.6638
MobileNetV2 | Train Loss: 0.854102373123169 | Train Accuracy: 0.6440839767456055 | Validation Loss: 0.7784525156021118 | Validation Accuracy: 0.6637930870056152

# -------------------- MobileNetV2: 3-fold -------------------- #
17/17 [==============================] - 4s 247ms/step - loss: 1.2018 - accuracy: 0.3053 - val_loss: 1.1838 - val_accuracy: 0.3966
Number of layers in the base model:  155
Epoch 2/2
17/17 [==============================] - 4s 263ms/step - loss: 0.9018 - 

# Ensemble

In [12]:
# tf.random.set_seed(SEED)
# hyperparams = {
#     'initial_epochs': 150,
#     'learning_rate': 1e-4,
#     'label_smoothing': 0.1,
# }
# experiment_name = f'Ensemble02'


# ensemble_model = create_ensemble_model()
# train_ds = ensemble_input(train_folds[3], ds_type='train')
# valid_ds = ensemble_input(valid_folds[3], ds_type='valid')
# ensemble_model = train_validate(ensemble_model, train_ds, valid_ds, hyperparams, experiment_name)


In [13]:
# test_ds = ensemble_input(test_ds, ds_type='train')
# predictions = ensemble_model.predict(test_ds)
# predicted_indices = tf.argmax(predictions, 1)
# predicted_labels = predicted_indices.numpy()
# img_ids = tfds.as_dataframe(test_ds_raw, test_info_raw)
# save_results('Ensemble02_submission.csv', img_ids, predicted_labels)
# print('done')

# Feature Extraction + Fine Tuning Script

In [10]:
data_folder = 'data'
raw_folder = 'raw'
processed_folder = 'processed'
dataset = 'mri_dataset'
train_folder = 'train'
test_folder = 'test'
train_label = 'train_label.csv'

SEED = 0

experiment_name = 'MobileNetV2'
result_save_path = experiment_name + '_' + 'submission.csv'
model_params = {
    'model_type': 'mobile',
    'head_type': 'standard',
    'image_shape': (512, 512, 3),
    'num_classes': 3,
}
base_hyperparams = {
    'train_batch_size': 64,
    'valid_batch_size': 64,
    'test_batch_size': 64,
    'num_epochs': 1,
    'learning_rate': 1e-4,
    'dropout': 0.2
}
fine_hyperparams = {
    'num_epochs': 1,
    'learning_rate': 1e-5,
    'fine_tune_at': 100,
}

tf.random.set_seed(SEED)

train_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[:{k}%]+train[{k+10}%:]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
valid_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[{k}%:{k+10}%]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
test_ds_raw, test_info_raw = tfds.load(
    name='mri_dataset', 
    split='test', 
    download=False, 
    shuffle_files=False, 
    as_supervised=False, 
    with_info=True,
    data_dir=data_folder
)

train_ds =  preprocess(train_folds[0], model_params, batch_size=base_hyperparams['train_batch_size'], ds_type='train')
valid_ds = preprocess(valid_folds[0], model_params, batch_size=base_hyperparams['valid_batch_size'], ds_type='valid')
test_ds = preprocess(test_ds_raw, model_params, batch_size=base_hyperparams['test_batch_size'], ds_type='test')

model = feature_extract_and_fine_tune(experiment_name, train_ds, valid_ds, model_params, base_hyperparams, fine_hyperparams)
predicted_labels = evaluate(model, test_ds)
img_ids = tfds.as_dataframe(test_ds_raw, test_info_raw)
save_results(img_ids, predicted_labels, result_save_path)

 1/17 [>.............................] - ETA: 0s - loss: 1.4278 - accuracy: 0.1562WARNING:tensorflow:From /home/zongsien/MedicalImageClassifier/env/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
17/17 [==============================] - 9s 548ms/step - loss: 1.2701 - accuracy: 0.2595 - val_loss: 1.0944 - val_accuracy: 0.4224
Number of layers in the base model:  155
Epoch 2/2
17/17 [==============================] - 5s 313ms/step - loss: 0.9223 - accuracy: 0.5716 - val_loss: 0.7705 - val_accuracy: 0.6638


In [17]:
# enhance images
# create dataset
# load data
# train
# evaluate